In [1]:
plate_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate1.csv"
plate_1_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate1_repeat.csv"
plate_2_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_1.csv"
plate_2_2 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_2.csv"
plate_2_3 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_3.csv"
plate_2_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_repeat.csv"
plate_2_repeat_96 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_repeat _96.csv"
list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listA.csv"
list_A_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listA_repeat.csv"
list_B = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listB.csv"
list_B_repeat_end = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listB_repeat _end.csv"
list_B_repeat_96 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listB_repeat _96.csv"
list_C = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listC.csv"

In [2]:
import pandas as pd
import numpy as np
import csv
import re
import string
from itertools import product

# define custom errors
class DataError(Exception):
    pass

class PlateSizeError(Exception):
    pass


# define well plate dimensions
plate_dimensions = {'96':(8, 12), '384':(16, 24)}
    
    
class FA:
    """Class used for the analysis of fluorescence anisotropy data.
    
    :param data_dict: A dictionary contaning data frames with pre-processed data and metadata
    :type data_dict: dict
    :param g_factor: G-factor
    :type g_factor: float """
    
    def __init__(self, data_dict, g_factor):
        self.data_dict = data_dict
        self.g_factor = g_factor
    
    
    @classmethod
    def read_in_envision(cls, csv_file, data_type='plate', size=384):
        
        """Returns a dictionary of data frames and g-factor needed to construct the class object. 
        
        :param csv_file: File path of the raw data file in .csv format
        :type csv_file: str
        :param data_type: Format in which the raw data was exported (plate or list), defaults to plate
        :type data_type: str
        :param size: Size of the well plate (384 or 96), defaults to 384
        :type size: int
        :return: A dictionary contaning data frames with pre-processed data and g-factor
        :rtype: dict, float """
        
        # ensure the plate size is 384 or 96
        if str(size) not in plate_dimensions:
            raise PlateSizeError('Invalid size of the well plate, should be 384 or 96.')
        
        # generate a list of well IDs for the pre-processed data frames
        row_letters = list(string.ascii_uppercase)[0: plate_dimensions[str(size)][0]]   # generate a list of letters for well IDs
        col_numbers = list(np.arange(1, plate_dimensions[str(size)][1] + 1).astype(str))   # generate a list of numbers for well IDs
        well_ids = ['%s%s' % (item[0], item[1]) for item in product(row_letters, col_numbers)]
        
        # try to read in data in plate format
        if data_type == 'plate':
            try:
                data_dict, g_factor = FA.read_in_plate(csv_file, well_ids)
                return cls(data_dict, g_factor)
            
            except (UnboundLocalError, IndexError, ValueError):
                raise DataError(f"Error occured during data read in. Check your file contains data in the 'plate' format and plate size is {size}.")
        
        # try to read in data in list format
        if data_type == 'list':
            try:
                data_dict, g_factor = FA.read_in_list(csv_file, well_ids)
                return cls(data_dict, g_factor)
            
            except (UnboundLocalError, IndexError):
                raise DataError("Error occured during data read in. Check your file contains data in the 'list' format.")
                
                
    def read_in_plate(csv_file, well_ids):
        
        """Reads the raw data file and finds the information needed to extract data. Passes those parameters to pre_process_plate function and executes it.
        Returns a tuple of two elemnts: dictionary of data frames and g-factor.

        :param csv_file: File path of the raw data file in .csv format
        :type csv_file: str
        :param well_ids: A list of well IDs for the pre-processed data frames
        :type well_ids: list
        :return: A tuple of dictionary of data frames and the g-factor 
        :rtype: tuple """
        
        with open(csv_file) as file:
            all_data_lines = list(csv.reader(file, delimiter=','))   # read the csv file and cast it into a list containing all lines

        blank_indexes = list(index for index, item in enumerate(all_data_lines) if item == [])   # list containing indices of all blank rows
        blanks = np.array(blank_indexes)   # convert the list of blank indices to a numpy array
        read_in_info = []   # list to store the tuples with parameters needed for pandas to read in the csv file

        for index, item in enumerate(all_data_lines):   # iterate over all lines in the csv file
            if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and re.search(r'Results for', all_data_lines[index + 9][0]) == None and re.findall(r"Formula", all_data_lines[index+1][10]) != ['Formula']:
                skiprows = index + 9   # Set the skiprows parameter for raw data table
                skiprows_meta = index + 1   # Set the skiprows parameter for metadata table
                end_of_data = blanks[blanks > skiprows].min()   # Calculate the end of data table by finding the smallest blank index after the beginning of data table
                read_in_info.append((skiprows, end_of_data - skiprows + 1, skiprows_meta))   # add the skiprows, caculated number of data lines and skiprows for metadata parameters to the list as a tuple
                data_format = 'plate1'

            if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and re.search(r'Results for', all_data_lines[index + 9][0]) != None:
                skiprows = index + 10
                skiprows_meta = index + 1
                end_of_data = blanks[blanks > skiprows].min()
                read_in_info.append((skiprows, end_of_data - skiprows, skiprows_meta))
                data_format = 'plate2'

            if item != [] and len(item) > 1 and re.findall(r"G-factor", item[0]) == ["G-factor"]:
                g_factor = float(item[4])   
        
        return FA.pre_process_plate(csv_file, read_in_info, data_format, well_ids), g_factor

    
    def pre_process_plate(csv_file, read_in_info, data_format, well_ids):    

        """Extracts the data and metadata from the csv file, processes it and returns a nested dictionary containing data and metadata for each repeat and channel.

        :param csv_file: File path of the raw data file in .csv format
        :type csv_file: str
        :param read_in_info: Tuples with read in parameters for each channel.
        :type read_in_info: list
        :param data_format: Plate type (plate1 or plate2)
        :type data_format: str
        :param well_ids: A list of well IDs for the pre-processed data frames
        :type well_ids: list
        :return: A dictionary containing data and metadata 
        :rtype: dict """ 
        
        data_frames = {}   # dictionary to store data frames
        counter = 1   # counter incremented by 0.5 to enable alternating labelling of data frames as 'p' or 's'

        for index, item in enumerate(read_in_info):   # iterate over all tuples in the list, each tuple contains skiprows, nrows and skiprows_meta for one channel 

            if data_format == 'plate1':   # raw data table does not have row and column names so 'names' parameter passed to omit the last column
                raw_data = pd.read_csv(csv_file, sep=',', names=col_numbers, index_col=False, engine='python', skiprows=item[0], nrows=item[1], encoding='utf-8')

            if data_format == 'plate2':   # raw data table has row and column names, so index_col=0 to set the first column as row labels
                raw_data = pd.read_csv(csv_file, sep=',', index_col=0, engine='python', skiprows=item[0], nrows=item[1], encoding='utf-8')
                raw_data.drop(raw_data.columns[-1], axis=1, inplace=True)   # delete the last column because it is empty

            # generate df for metadata (number of rows of metadata table is always 1) and convert measurement time into datetime object   
            metadata = pd.read_csv(csv_file, sep=',', engine='python', skiprows=item[2], nrows=1, encoding='utf-8').astype({'Measurement date': 'datetime64[ns]'})
            # convert and reshape data frame into 1D array
            data_as_array = np.reshape(raw_data.to_numpy(), (int(well_ids), 1)) 

            if counter % 1 == 0: 
                new_data = pd.DataFrame(data=data_as_array, index=well_ids, columns=['p'])   # generate new 384 (or 96) by 1 data frame with p channel data
                data_frames[f'repeat_{int(counter)}'] = {'metadata':metadata, 'data': {'p': new_data, 's':''}}   # add p channel data and metadata dfs to dictionary

            if counter % 1 != 0:
                new_data = pd.DataFrame(data=data_as_array, index=well_ids, columns=['s'])   # generate new 384 (or 96) by 1 data frame with s channel data
                data_frames[f'repeat_{int(counter-0.5)}']['data']['s'] = new_data   # add s channel data to dictionary

            counter = counter + 0.5
        
        return data_frames


    def read_in_list(csv_file, well_ids):

        """Reads the raw data file and extracts the data and metadata. Passes the raw data to pre_process_list function and executes it.
        Returns a tuple of two elemnts: dictionary of data frames and g-factor.

        :param csv_file: File path of the raw data file in .csv format
        :type csv_file: str
        :param well_ids: A list of well IDs for the pre-processed data frames
        :type well_ids: list
        :return: A tuple of dictionary of data frames and the g-factor
        :rtype: tuple """

        with open(csv_file) as file:  
            all_data_lines = list(csv.reader(file, delimiter=',')) # read the csv file and cast it into a list containing all lines
 
        blank_indexes = list(index for index, item in enumerate(all_data_lines) if item == [])   # list containing indexes of all blank rows
        blanks = np.array(blank_indexes)   # convert the list of blank indexes to a numpy array
        
        # set the skiprows to be initially greater than the total number of lines in the file to enable the evaluation of if statement until the 'skiprows' parameter is found
        skiprows = len(all_data_lines) + 1
        
        # iterate over all lines to find beggining of the data table ('skiprows') and determine the format of data  (list A, B, or C)
        for index, item in enumerate(all_data_lines):   
            if item != [] and len(item) == 1 and re.findall(r"Plate information", item[0]) == ["Plate information"]:
                skiprows_meta = index + 1
                end_of_metadata = blanks[blanks > skiprows_meta].min()   # find the end of metadata by finding the smallest blank index after the beginning of metadata
                
            if item != [] and len(item) >= 2 and re.findall(r"PlateNumber", item[0]) == ['PlateNumber'] and re.findall(r"PlateRepeat", item[1]) == ['PlateRepeat']:   # find line number with the beggining of the data
                skiprows = index - 1
                data_format = 'listA'
                end_of_data = blanks[blanks > skiprows].min()

            if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate'] and re.findall(r"Barcode", item[1]) == ['Barcode']:   # find line number with the beggining of the data
                skiprows = index
                data_format = 'listB'
                end_of_data = blanks[blanks > skiprows].min()

            if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate']  and re.findall(r"Well", item[1]) == ['Well']:
                skiprows = index
                data_format = 'listC'
                end_of_data = blanks[blanks > skiprows].min()

            if item != [] and re.findall(r"G-factor", item[0]) == ["G-factor"]:   # find the g factor
                g_factor = float(item[4])

        nrows = end_of_data - skiprows - 1   # calculate the length of data table
        nrows_meta = end_of_metadata - skiprows_meta - 1   # calucalte the length of metadata table (number of rows depends on the number of repeats)

        raw_data = pd.read_csv(csv_file, sep=',', engine='python', skiprows=skiprows, nrows=nrows, encoding='utf-8')
        raw_metadata = pd.read_csv(csv_file, sep=',', engine='python', skiprows=skiprows_meta, nrows=nrows_meta, encoding='utf-8')

        return FA.pre_process_list(raw_data, raw_metadata, data_format, well_ids), g_factor


    def pre_process_list(raw_data, raw_metadata, data_format, well_ids):

        """Extracts the data and metadata for each channel and repeat from the raw data and raw metadata 
        and returns a nested dictionary containing data and metadata for each repeat and channel.

        :param raw_data: Data frame containing raw data
        :type raw_data: pandas data frame
        :param raw_metadata: Data frame containing raw metadata
        :type raw_metadata: pandas data frame
        :param data_format: Type of list (listA, listB, or listC)
        :type data_format: str
        :param well_ids: A list of well IDs for the pre-processed data frames
        :type well_ids: list
        :return: A dictionary containing data and metadata
        :rtype: dict"""

        # remove the '0' from middle position of well numbers (A01 -> A1), done by reassigning the 'Well' column to a Series containing modified well numbers
        raw_data['Well'] = raw_data['Well'].apply(lambda x: x[0] + x[2] if x[1] == '0' else x)
        
        data_frames = {}   # dictionary to store data frames
        repeats = list(raw_metadata['Repeat'].to_numpy())   # generate a list with repeats based on the metadata table, e.g. for 3 repeats -> [1,2,3]

        for index, repeat in enumerate(repeats):   # iterate over the number of repeats
            if data_format == 'listA':
                groupped_data = raw_data.groupby(raw_data.PlateRepeat).get_group(repeat)   # group and extract the data by the plate repeat column, i.e. in each iteration get data only for the current repeat 

                p_groupped = groupped_data.iloc[::3, :]   # extract data only for the p channel, i.e. each third row starting from the first row
                s_groupped = groupped_data.iloc[1::3, :]   # extract data only for the s channel, i.e. each third row starting from the second row

                p_raw_data = p_groupped[['Well', 'Signal']]   # extract only the two relevant columns
                s_raw_data = s_groupped[['Well', 'Signal']]   # for each channel

            if data_format in ['listB', 'listC']: 
                # the column naming is different for the first repeat ('Signal'), then it's 'Signal.1', 'Signal.2', etc.
                if repeat == 1: 
                    p_raw_data = raw_data[['Well', 'Signal']]   
                    s_raw_data = raw_data[['Well', f'Signal.{repeat}']]
                else:
                    p_raw_data = raw_data[['Well', f'Signal.{repeat + index - 1}']]   # the column cotntaining data to be extracted is calculated in each iteration
                    s_raw_data = raw_data[['Well', f'Signal.{repeat + index}']]
            
            # create an empty df with no columns and indexes matching the plate size
            indexes = pd.DataFrame(well_ids, columns=['Wells'])
            empty_frame = indexes.set_index('Wells')
            
            p_raw_data.set_index('Well', inplace=True)   # set the row indexes as the well numbers
            p_raw_data.set_axis(['p'], axis=1, inplace=True)   # rename the 'Signal' column to 'p'
            p_data = empty_frame.join(p_raw_data)   # join the raw data df to an empty frame based on the indexes, assigns 'NaN' to indexes not present in the raw data table
            
            s_raw_data.set_index('Well', inplace=True)
            s_raw_data.set_axis(['s'], axis=1, inplace=True)
            s_data = empty_frame.join(s_raw_data)
    
            metadata = raw_metadata.iloc[[repeat-1]].astype({'Measurement date': 'datetime64[ns]'})   # extract the row with metadata relevant for each repeat and covert date and time into a datetime object
            data_frames[f'repeat_{repeat}'] = {'metadata': metadata, 'data': {'p': p_data, 's': s_data}}   # add data frames to the dictionary

        return data_frames

In [3]:
my_data = FA.read_in_envision(csv_file=list_A_repeat, data_type='list', size=384)

In [4]:
my_data.g_factor

1.15

In [5]:
my_data3 = FA.read_in_envision(csv_file=list_B, data_type='list', size=96)

In [6]:
my_data3.data_dict

{'repeat_1': {'metadata':    Plate  Repeat  Barcode  Measured height  Chamber temperature at start  \
  0      1       1      NaN             14.4                         18.98   
  
     Chamber temperature at end  Humidity at start  Humidity at end  \
  0                          19               61.5             61.1   
  
     Ambient temperature at start  Ambient temperature at end  \
  0                         18.98                       19.39   
  
       Measurement date  Unnamed: 11  
  0 2020-11-17 13:30:28          NaN  ,
  'data': {'p':                 p
   Wells            
   A1     18890869.0
   A2     27377968.0
   A3     15983855.0
   A4     21051671.0
   A5     22837285.0
   ...           ...
   H8            NaN
   H9            NaN
   H10           NaN
   H11           NaN
   H12           NaN
   
   [96 rows x 1 columns],
   's':                 s
   Wells            
   A1     20668058.0
   A2     29442193.0
   A3     18411616.0
   A4     23222549.0
   A5     250

In [7]:
my_data.data_dict['repeat_1']['data']['s']

,s
Wells,
A1,20469296.0
A2,29296716.0
A3,18210982.0
A4,23159988.0
A5,24960618.0
...,...
P20,NaN
P21,NaN
P22,NaN


In [8]:
my_data3.g_factor

1.15

In [9]:
my_data3.data_dict['repeat_1']['data']['p']

,p
Wells,
A1,18890869.0
A2,27377968.0
A3,15983855.0
A4,21051671.0
A5,22837285.0
...,...
H8,NaN
H9,NaN
H10,NaN


In [10]:
my_data2 = FA.read_in_envision(csv_file=list_B_repeat_96, data_type='list', size=96)

In [11]:
my_data2.g_factor

1.0

In [12]:
my_data2.data_dict['repeat_1']['metadata']

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Measurement date,Unnamed: 11
0,1,1,NaN,14.52,18.78,18.68,61.8,62.3,18.88,18.98,2020-11-17 13:39:13,NaN


In [13]:
my_data2.data_dict['repeat_2']['data']['p'].iloc[0:50]

,p
Wells,
A1,352962.0
A2,357763.0
A3,541382.0
A4,NaN
A5,NaN
A6,NaN
A7,NaN
A8,NaN
A9,NaN
